In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.express as px
from wordcloud import WordCloud
from collections import Counter
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 시가총액 상위 300개 기업리스트 (미래에셋 대우 API 미확인으로, 2021.09.12 기준 시총 상위 500개 중 300개 기업 사용)
df_4 = pd.read_csv("/content/drive/MyDrive/KOSPI_시가총액_상위500.csv")
lists_1 = df_4.loc[:300]
lists_2 = lists_1[['종목코드','종목명']]

In [ ]:
lists_2

,종목코드,종목명
0,005930,삼성전자
1,000660,SK하이닉스
2,035420,NAVER
3,207940,삼성바이오로직스
4,005935,삼성전자우
...,...,...
296,034830,한국토지신탁
297,068400,SK렌터카
298,037560,LG헬로비전
299,006390,한일현대시멘트


In [ ]:
stock_score = []
for jongmok,code in zip(lists_2['종목명'],lists_2['종목코드']):
    url = 'https://finance.naver.com/item/coinfo.naver?code={}'.format(code)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    try:
        info_list = soup.find('table', attrs = {'class':'rwidth'})
        score = info_list.find_all('span')[1].text
        if score[:4] != 'N/A':
            stock_score.append([jongmok, float(score[:4])])
        else:
            stock_score.append([jongmok, 0.00])
    except:
        stock_score.append([jongmok, 0.00])

In [ ]:
stock_score

[['삼성전자', 4.0],
 ['SK하이닉스', 4.0],
 ['NAVER', 4.0],
 ['삼성바이오로직스', 4.0],
 ['삼성전자우', 4.0],
 ['카카오', 4.0],
 ['LG화학', 4.0],
 ['삼성SDI', 4.0],
 ['현대차', 4.0],
 ['셀트리온', 4.0],
 ['기아', 4.0],
 ['카카오뱅크', 3.67],
 ['POSCO', 4.0],
 ['현대모비스', 3.95],
 ['삼성물산', 3.9],
 ['LG전자', 4.0],
 ['SK이노베이션', 3.81],
 ['SK바이오사이언스', 4.0],
 ['SK텔레콤', 4.0],
 ['크래프톤', 4.0],
 ['LG생활건강', 4.0],
 ['KB금융', 4.0],
 ['신한지주', 4.0],
 ['SK', 4.0],
 ['SK아이이테크놀로지', 4.0],
 ['HMM', 3.33],
 ['한국전력', 3.36],
 ['LG', 3.86],
 ['삼성생명', 4.0],
 ['삼성전기', 4.0],
 ['엔씨소프트', 3.72],
 ['하나금융지주', 4.0],
 ['삼성에스디에스', 4.0],
 ['포스코케미칼', 3.88],
 ['아모레퍼시픽', 3.53],
 ['KT&G', 4.0],
 ['대한항공', 3.67],
 ['S-Oil', 3.9],
 ['삼성화재', 4.0],
 ['넷마블', 4.0],
 ['두산중공업', 0.0],
 ['하이브', 3.93],
 ['고려아연', 3.92],
 ['SK바이오팜', 4.0],
 ['한온시스템', 3.62],
 ['KT', 4.0],
 ['롯데케미칼', 3.9],
 ['한국조선해양', 4.0],
 ['한화솔루션', 3.88],
 ['우리금융지주', 4.0],
 ['기업은행', 3.81],
 ['LG디스플레이', 3.94],
 ['현대제철', 4.0],
 ['현대글로비스', 3.93],
 ['SKC', 3.93],
 ['CJ제일제당', 4.0],
 ['LG유플러스', 4.0],
 ['금호석유', 3.85],
 ['강원랜드'

In [ ]:
url = 'https://finance.naver.com/item/coinfo.naver?code=005930'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
info_list = soup.find('table', attrs = {'class':'rwidth'})
info_list.find_all('span')[1].text

'4.00매수'

In [ ]:
info_list

In [ ]:
stock_score = pd.DataFrame(stock_score, columns = ['종목명', '점수'])

In [ ]:
stock_score[['점수']].value_counts()

점수  
4.00    170
0.00     52
3.00      7
3.86      5
3.83      5
3.93      5
3.92      5
3.90      4
3.33      4
3.67      4
3.88      3
3.62      3
4.06      2
3.50      2
3.53      2
3.20      2
3.60      2
3.75      2
3.80      2
3.81      2
3.85      2
3.91      2
3.94      2
3.95      2
3.72      1
3.71      1
3.82      1
3.56      1
3.89      1
3.36      1
3.25      1
3.12      1
2.00      1
4.33      1
dtype: int64

In [ ]:
stock_score_fnguide = []
for jongmok,code in zip(lists_2['종목명'],lists_2['종목코드']):
    url = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{}'.format(code)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    try:
        info_list = soup.find('div', attrs = {'id' : 'svdMainGrid9'})
        score = info_list.find('tbody').find_all('td')[0].text
        try:
            stock_score_fnguide.append([jongmok, float(score)])
        except:
            stock_score_fnguide.append([jongmok, 0.00])
    except:
        stock_score_fnguide.append([jongmok, 0.00])

In [ ]:
url = 'https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A005930'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
info_list = soup.find('div', attrs = {'id' : 'svdMainGrid9'})
info_list.find('tbody').find_all('td')[0].text

'4.0'

In [ ]:
stock_score_fnguide

[['삼성전자', 4.0],
 ['SK하이닉스', 4.0],
 ['NAVER', 4.0],
 ['삼성바이오로직스', 4.0],
 ['삼성전자우', 0.0],
 ['카카오', 4.0],
 ['LG화학', 4.0],
 ['삼성SDI', 4.0],
 ['현대차', 4.0],
 ['셀트리온', 4.0],
 ['기아', 4.0],
 ['카카오뱅크', 3.7],
 ['POSCO', 4.0],
 ['현대모비스', 4.0],
 ['삼성물산', 3.9],
 ['LG전자', 4.0],
 ['SK이노베이션', 3.8],
 ['SK바이오사이언스', 4.0],
 ['SK텔레콤', 4.0],
 ['크래프톤', 4.0],
 ['LG생활건강', 4.0],
 ['KB금융', 4.0],
 ['신한지주', 4.0],
 ['SK', 4.0],
 ['SK아이이테크놀로지', 4.0],
 ['HMM', 3.4],
 ['한국전력', 3.4],
 ['LG', 3.8],
 ['삼성생명', 4.0],
 ['삼성전기', 4.0],
 ['엔씨소프트', 3.7],
 ['하나금융지주', 4.0],
 ['삼성에스디에스', 4.0],
 ['포스코케미칼', 3.9],
 ['아모레퍼시픽', 3.5],
 ['KT&G', 4.0],
 ['대한항공', 3.8],
 ['S-Oil', 3.9],
 ['삼성화재', 4.0],
 ['넷마블', 4.0],
 ['두산중공업', 0.0],
 ['하이브', 3.9],
 ['고려아연', 3.9],
 ['SK바이오팜', 4.0],
 ['한온시스템', 3.6],
 ['KT', 4.0],
 ['롯데케미칼', 3.9],
 ['한국조선해양', 4.0],
 ['한화솔루션', 3.9],
 ['우리금융지주', 4.0],
 ['기업은행', 3.8],
 ['LG디스플레이', 3.9],
 ['현대제철', 4.0],
 ['현대글로비스', 3.9],
 ['SKC', 3.9],
 ['CJ제일제당', 4.0],
 ['LG유플러스', 4.0],
 ['금호석유', 3.9],
 ['강원랜드', 4.0],
 ['현대건설', 4

In [ ]:
stock_score = []
for i in range(6):
    url = 'https://kr.investing.com/stock-screener/?sp=country::11|sector::a|industry::a|equityType::a|RSI::10.827,100|eq_beta::-3.32,4.07|eq_one_year_return::-99.64,12015.38|month_change::-99.64,296.32|STOCH::0,99.364|MACD::-21227.714,36107.02%3Ceq_market_cap;{}'.format(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    try:
        info_list = soup.find('table', attrs = {'class':'rwidth'})
        score = info_list.find_all('span')[1].text
        if score[:4] != 'N/A':
            stock_score.append([jongmok, float(score[:4])])
        else:
            stock_score.append([jongmok, 0.00])
    except:
        stock_score.append([jongmok, 0.00])

In [5]:
url = 'https://kr.investing.com/stock-screener/?sp=country::11|sector::a|industry::a|equityType::a|RSI::10.827,100|eq_beta::-3.32,4.07|eq_one_year_return::-99.64,12015.38|month_change::-99.64,296.32|STOCH::0,99.364|MACD::-21227.714,36107.02%3Ceq_market_cap;1'
res = requests.get(url, headers = headers)
soup = BeautifulSoup(res.text, 'lxml')
soup

<!DOCTYPE HTML>
<html class="kr" dir="ltr" geo="US" lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml" xmlns:schema="http://schema.org/">
<head>
<script type="text/javascript">
    function OptanonWrapper() { }

    // temp for checks
    </script>
<style>
	.ot-floating-button {
		z-index:12 !important;
	}
</style>
<script type="text/javascript">
        window.helpers = {
            'getCookie': function (k){var v="; "+document.cookie;var p=v.split("; "+k+"=");if(p.length===2)return p.pop().split(";").shift();else return null},
            'setCookie': function (k,v,d){var e='';if(typeof d!=='undefined'){var n=new Date;n.setTime(n.getTime()+864e5*d);e+=';expires="';e+=n.toGMTString();e+='"';}document.cookie=k+'='+v+';path=/' + e},
            'delCookie': function (k){document.cookie=k+'=;path=/;expires=-1'},
            'isNumber': function (n){return !isNaN(n)&&typeof n!==undefined&&n!==null&&n!==''},
            'removeClass': function (e,c){if(document.documentElement.c

In [4]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

In [16]:
info_list = soup.find_all('table', attrs = {'id':'resultsTable'})[0]
info_list

<table class="displayNone genTbl openTbl resultsStockScreenerTbl elpTbl " id="resultsTable">
<thead>
<tr>
<th class="flag"> </th>
<th class="symbol left js-sortable pointer" data-column-name="name_trans"><i title="종목">종목</i><span class="headerSortDefault"></span></th><th class="left js-sortable pointer" data-column-name="viewData.symbol"><i title="기호">기호</i><span class="headerSortDefault"></span></th><th class="left displayNone js-sortable pointer" data-column-name="exchange_trans"><i title="거래소">거래소</i><span class="headerSortDefault"></span></th><th class="left displayNone js-sortable pointer" data-column-name="sector_trans"><i title="부문">부문</i><span class="headerSortDefault"></span></th><th class="left displayNone js-sortable pointer" data-column-name="industry_trans"><i title="산업">산업</i><span class="headerSortDefault"></span></th><th class=" js-sortable pointer" data-column-name="last"><i title="종가">종가</i><span class="headerSortDefault"></span></th><th class=" js-sortable pointer" d

In [ ]:
stock_score = []
for jongmok,code in zip(lists_2['종목명'],lists_2['종목코드']):
    url = 'https://finance.naver.com/item/coinfo.naver?code={}'.format(code)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'lxml')
    try:
        info_list = soup.find('table', attrs = {'id':'resultsTable'}).find('tbody')
        score = info_list.find_all('span')[1].text
        if score[:4] != 'N/A':
            stock_score.append([jongmok, float(score[:4])])
        else:
            stock_score.append([jongmok, 0.00])
    except:
        stock_score.append([jongmok, 0.00])